# README.

This notebook runs the evaluation on the cluster.

# Imports.

In [1]:
from azureml.core import Workspace, Dataset
from azureml.core import Experiment

# Connect to azure workspace.

In [2]:
workspace = Workspace.from_config()
workspace

Workspace.create(name='cgm-ml-dev', subscription_id='9b82ecea-6780-4b85-8acf-d27d79028f07', resource_group='cgm-ml-dev')

# Get the experiment.

In [3]:
from azureml.core import Experiment
experiment_name = "s1-gapnet-weight"
experiment = Experiment(workspace=workspace, name=experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
s1-gapnet-weight,cgm-ml-dev,Link to Azure Machine Learning studio,Link to Documentation


# Find/create a compute target.

In [4]:
import os
import urllib
import shutil
import azureml
 
from azureml.core import Experiment
from azureml.core import Workspace, Run

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "tristan-gpu"

try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_NC6', 
       max_nodes=4)
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
compute_target

Found existing compute target


AmlCompute(workspace=Workspace.create(name='cgm-ml-dev', subscription_id='9b82ecea-6780-4b85-8acf-d27d79028f07', resource_group='cgm-ml-dev'), name=tristan-gpu, id=/subscriptions/9b82ecea-6780-4b85-8acf-d27d79028f07/resourceGroups/cgm-ml-dev/providers/Microsoft.MachineLearningServices/workspaces/cgm-ml-dev/computes/tristan-gpu, type=AmlCompute, provisioning_state=Succeeded, location=westeurope, tags=None)

# Get the dataset for evaluation.

In [5]:
#dataset_name = "cgmmldevpremium-SampleDataset-Example" # Tiny dataset.
dataset_name = "cgmmldevpremium-SampleDataset" # Full dataset.
dataset = workspace.datasets[dataset_name]
dataset

{
  "source": [
    "('premiumfileshare', '2019_09_23_11_48_06-30K_samples/**')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "7fac12d6-3069-456a-bfa2-29141f2e4f07",
    "name": "cgmmldevpremium-SampleDataset",
    "version": 2,
    "workspace": "Workspace.create(name='cgm-ml-dev', subscription_id='9b82ecea-6780-4b85-8acf-d27d79028f07', resource_group='cgm-ml-dev')"
  }
}

# Push the evaluation source code to Azure.

In [6]:
from azureml.train.dnn import TensorFlow
TensorFlow.get_supported_versions()

['1.10', '1.12', '1.13', '2.0']

In [7]:
pip_packages = [
    "azureml-dataprep[fuse,pandas]",
    "glob2"
]

script_params = {
    "--run_id": "s1-gapnet-weight_1579792118_0c294b2b" # The run that contains the model.
}

# Create the estimator.
estimator = TensorFlow(
    source_directory="code",
    compute_target=compute_target,
    entry_script="evaluate.py",
    script_params=script_params,
    use_gpu=True,
    framework_version="2.0",
    inputs=[dataset.as_named_input("dataset").as_mount()],
    pip_packages=pip_packages
)

# Set compute target.
estimator.run_config.target = compute_target

# Run the experiment.
run = experiment.submit(estimator)

# Show outpus.
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…